In [1]:
import numpy as np
import pickle
from TrainVoxelmorph import VoxelmorphTF
#from trainPytorch import VoxelmorphTorch
from DataHandler import DataHandler
import SimpleITK as sitk
import os
#import tensorflow as tf
#import torch as th
from DataHandler import DataHandler
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [16]:
dataSet = 'native'
dhCT = DataHandler(val_images=12)
dhCT.get_synthetic_data(
    fixed_path=f'/home/cschellenberger/datam2olie/synthetic/{dataSet}/t3/Synthetic_CT/',
    moving_path=f'/home/cschellenberger/datam2olie/synthetic/{dataSet}/t1/Synthetic_CT/')
dhMR = DataHandler(val_images=12)
dhMR.get_synthetic_data(
    fixed_path=f'/home/cschellenberger/datam2olie/synthetic/{dataSet}/t3/Synthetic_CT/',
    moving_path=f'/home/cschellenberger/datam2olie/synthetic/{dataSet}/t1/Synthetic_MR/')
moving_image_paths_MR = dhMR.x_val
fixed_image_paths_MR = dhMR.y_val
moving_image_paths_CT = dhCT.x_val
fixed_image_paths_CT = dhCT.y_val
for idx in range(1):
    fixed_image = sitk.ReadImage(fixed_image_paths_MR[idx])
    moving_image_MR = sitk.ReadImage(moving_image_paths_MR[idx])
    moving_image_CT = sitk.ReadImage(moving_image_paths_CT[idx])
    moving_image_MR.SetSpacing(fixed_image.GetSpacing())
    moving_image_CT.SetSpacing(fixed_image.GetSpacing())
    moving_image_MR.SetOrigin(fixed_image.GetOrigin())
    moving_image_CT.SetOrigin(fixed_image.GetOrigin())
    #print(fixed_image.GetOrigin())
    #print(moving_image.GetOrigin())

In [3]:
from pyM2aia import M2aiaOnlineHelper
M2aiaHelper = M2aiaOnlineHelper("ipynbViewer", "jtfc.de:5050/m2aia/m2aia-no-vnc:with_exit", "8899")
with M2aiaHelper as helper:
    helper.show({"Moving_MR": moving_image_MR, "Moving_CT": moving_image_CT, "Fixed": fixed_image})

You can find your images @  http://141.19.142.80:8899



In [2]:
def get_moved_points(points: np.array, displacement: sitk.Image) -> np.array:
    displacement_copy = displacement.__copy__()
    displacement_transform = sitk.DisplacementFieldTransform(displacement_copy)
    moved_points = [displacement_transform.TransformPoint(point) for point in points]
    return moved_points

def get_landmarks(fixed_image_path: str, indexing: str = 'zyx', preResample = False):
    model_name = os.path.basename(fixed_image_path).replace('_atn_3.nrrd', '')
    if preResample: 
        loaded_points = pickle.load(open(f'/home/cschellenberger/Documents/vectorPickles/{model_name}_vec_frame1_to_frame2.p', "rb"))
        moving_landmarks = np.array([(float(loaded_points[idx]['1X']), float(loaded_points[idx]['1Y']), float(loaded_points[idx]['1Z'] * 2)) for (idx, _) in enumerate(loaded_points)])
        fixed_landmarks = np.array([(float(loaded_points[idx]['2X']), float(loaded_points[idx]['2Y']), float(loaded_points[idx]['2Z'] * 2)) for (idx, _) in enumerate(loaded_points)])
    else: 
        loaded_points = pickle.load(open(f'/home/cschellenberger/Documents/vectorPickles/CT_points_t1_t3_withRegion/{model_name}_idx.p', "rb"))
        moving_landmarks = np.array(loaded_points['t1'])
        fixed_landmarks = np.array(loaded_points['t3'])
    regions = np.array(loaded_points['Region'])
    if indexing == 'xzy':
        # swap columns because numpy and vxm use zyx indexing and the data uses xyz indexing
        moving_landmarks[:, [1, 2]] = moving_landmarks[:, [2, 1]]
        fixed_landmarks[:, [1, 2]] = fixed_landmarks[:, [2, 1]]
    elif indexing == 'zyx':
        # swap columns because numpy and vxm use zyx indexing and the data uses xyz indexing
        moving_landmarks[:, [0, 2]] = moving_landmarks[:, [2, 0]]
        fixed_landmarks[:, [0, 2]] = fixed_landmarks[:, [2, 0]]
    elif indexing == 'yxz':
        # swap columns because numpy and vxm use zyx indexing and the data uses xyz indexing
        moving_landmarks[:, [0, 1]] = moving_landmarks[:, [1, 0]]
        fixed_landmarks[:, [0, 1]] = fixed_landmarks[:, [1, 0]]
    else: assert indexing == 'xyz', f'indexing can only be xyz or zyx. Got: {indexing}'
    return moving_landmarks.astype(np.float64), fixed_landmarks.astype(np.float64), regions

In [4]:
dh = DataHandler(val_images=12)
dh.get_synthetic_data(
    fixed_path='/home/cschellenberger/datam2olie/synthetic/orig/t3/Synthetic_CT/',
    moving_path='/home/cschellenberger/datam2olie/synthetic/orig/t1/Synthetic_MR/')
idx = 0
moving_image_paths = dh.x_val
fixed_image_paths = dh.y_val
livers = {}
for idx in range(1):
    fixed_image = sitk.ReadImage(fixed_image_paths[idx])
    moving_image = sitk.ReadImage(moving_image_paths[idx])
    moving_landmarks, fixed_landmarks, regions = get_landmarks(fixed_image_paths[idx], indexing='xyz', preResample= False)
    err = moving_landmarks - fixed_landmarks
    err = err * 1.8
    err = np.linalg.norm(err, axis = 1)
    err = [err[i] for i in range(len(err)) if 'liver' in regions[i]]


In [5]:
dh = DataHandler(val_images=12)
dh.get_synthetic_data(
    fixed_path='/home/cschellenberger/Documents/newT3Resample1_8',
    moving_path='/home/cschellenberger/Documents/T1ResampledSpacing1_8')
i = 0
moving_image_paths = dh.x_val
fixed_image_paths = dh.y_val
livers = {}
for i in range(1):
    fixed_image = sitk.ReadImage(fixed_image_paths[i])
    moving_image = sitk.ReadImage(moving_image_paths[i])
    moving_landmarks, fixed_landmarks, regions = get_landmarks(fixed_image_paths[i], indexing='xyz')
    liver = [0 if regions[idx] != 'known_vector_liver' else 1 for idx in range(len(fixed_landmarks))]
    liver_fixed = sitk.Image(fixed_image.GetSize(), sitk.sitkInt32)
    liver_fixed.SetOrigin(fixed_image.GetOrigin())
    liver_fixed.SetSpacing(fixed_image.GetSpacing())
    liver_moving = sitk.Image(fixed_image.GetSize(), sitk.sitkInt32)
    liver_moving.SetOrigin(fixed_image.GetOrigin())
    liver_moving.SetSpacing(fixed_image.GetSpacing())
    for idx, point in enumerate(fixed_landmarks):
        try: liver_fixed.SetPixel(int(point[0]), int(point[1]), int(point[2]), liver[idx])
        except: continue
    for idx, point in enumerate(moving_landmarks):
        try: liver_moving.SetPixel(int(point[0]), int(point[1]), int(point[2]), liver[idx])
        except: continue
    weights_path = '/home/cschellenberger/Documents/scripts/models/synthetic/newT3_best_localmi_reg001_08_1000_st43_lr3e-05_bat1/bestWeights.h5'
    device = '/cpu:0'
    nb_features = [[16, 32, 32, 32], [32, 32, 32, 32, 32, 16, 16]]
    imgReg = VoxelmorphTF(weights_path, fixed_image, nb_features, 1)
    moved_img, displacement_np, time = imgReg.register_images(moving_image, fixed_image, device)
    displacement = displacement_np.squeeze() * 1.8
    displacement[:,:,:,[0,2]] = displacement[:,:,:,[2,0]]
    displacement = sitk.GetImageFromArray(displacement.astype(np.float64), isVector=True)
    #displacement.SetOrigin(fixed_image.GetOrigin())
    #displacement.SetSpacing(fixed_image.GetSpacing())
    moved_landmarks = get_moved_points(fixed_landmarks, displacement)
    liver = [0 if regions[idx] != 'known_vector_liver' else 1 for idx in range(len(moved_landmarks))]
    liver_moved = sitk.Image(fixed_image.GetSize(), sitk.sitkInt32)
    for idx, point in enumerate(moved_landmarks):
        try: liver_moved.SetPixel(int(point[0]), int(point[1]), int(point[2]), liver[idx])
        except: continue
    liver_moved.SetOrigin(fixed_image.GetOrigin())
    liver_moved.SetSpacing(fixed_image.GetSpacing())
    livers[f'liver_fixed{i}'] = liver_fixed
    livers[f'liver_moving{i}'] = liver_moving
    livers[f'fixed{i}'] = fixed_image
    livers[f'moving{i}'] = moving_image
    livers[f'moved{i}'] = moved_img
    livers[f'liver_moved{i}'] = liver_moved
#pickle.dump(livers, open("./liverSegments.p", "wb"))

In [40]:
err = moving_landmarks - moved_landmarks
err = np.linalg.norm(err, axis = 1)
print(err.mean())

8.724669438701286


In [43]:
bdi = sitk.BinaryDilateImageFilter()
bdi.SetForegroundValue(1)
livers['movedbdi'] = bdi.Execute(liver_moved)
livers['fixedbdi'] = bdi.Execute(liver_fixed)
livers['movingbdi'] = bdi.Execute(liver_moving)

In [7]:
from pyM2aia import M2aiaOnlineHelper
#livers = pickle.load(open("./liverSegments.p", "rb"))
M2aiaHelper = M2aiaOnlineHelper("ipynbViewer", "jtfc.de:5050/m2aia/m2aia-no-vnc:with_exit", "8899")
with M2aiaHelper as helper:
    helper.show(livers)

You can find your images @  http://141.19.142.80:8899

